FEATURES: title, preprocessed content, publisherId, length_preprocessed_content

MODEL: Logistic Regression and OneClassSVM

In [1]:
import psycopg2
import pandas as pd
import json
import pandas.io.sql as sqlio
# preprocess
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import string
# model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import sklearn.metrics as metrics

In [2]:
%config InlineBackend.figure_format = 'retina'
from pathlib import Path
import time
import pickle
from contextlib import contextmanager
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import seaborn as sns
from matplotlib import pyplot as plt
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

In [3]:
train_data=pd.read_csv("train_data3.csv")
train_data.head()

,index,id,content,title,preprocessed_content,label,hierarchy,hierarchy_#tiers,tier1,tier2,tier3,tier4
0,133,https://www.whowhatwear.com/5-looks-that-put-b...,After seeing burgundy boots on some of our fav...,5 Looks That Put Burgundy Boots On Our Must-Ha...,seeing burgundy boot favorite street style blo...,Womens\ Casual\ Wear,"[{'Style & Fashion': {""Women's Fashion"": {""Wom...",4,Style & Fashion,Womens Fashion,Womens Clothing,Womens Casual Wear
1,189,https://www.wisebread.com/what-you-need-to-kno...,"According to a September 29, 2009 Fidelity Inv...",How to Make the Most of Your 401K,according september fidelity investments study...,Personal\ Investing,[{'Personal Finance': 'Personal Investing'}],2,Personal Finance,Personal Investing,NaN,NaN
2,401,https://www.theturekclinic.com/blog/mindful-ea...,"“Thou shouldst eat to live; not live to eat,” ...",Mindful Eating,thou shouldst eat live live eat said socrates ...,Mens\ Health,"[{'Healthy Living': ""Men's Health""}]",2,Healthy Living,Mens Health,NaN,NaN
3,198,https://www.news-medical.net/news/20190611/Ear...,Researchers continue to dig for molecular clue...,Early life adversity and high levels of FKBP5 ...,researchers continue dig molecular clue better...,Hormonal\ Disorders,[{'Medical Health': {'Diseases and Conditions'...,4,Medical Health,Diseases and Conditions,Endocrine and Metabolic Diseases,Hormonal Disorders
4,24,https://hobbylark.com/card-games/Top-Strongest...,"As any duelist knows, the vast majority of Yu-...",Top 10 Strongest (Highest ATK) Monsters in Yu-...,duelist know vast majority yu gi oh match end ...,Card\ Games,[{'Hobbies & Interests': {'Games and Puzzles':...,3,Hobbies & Interests,Games and Puzzles,Card Games,NaN


In [4]:
train_data['tier1'].value_counts()

Sports                      927
Business and Finance        900
Technology & Computing      700
Style & Fashion             644
Personal Finance            600
Travel                      578
Hobbies & Interests         521
Medical Health              500
Education                   490
Books and Literature        365
Music and Audio             300
Automotive                  300
Pop Culture                 300
Movies                      279
Events and Attractions      271
Family and Relationships    200
Science                     200
Food & Drink                200
Real Estate                 200
Healthy Living              200
News and Politics           100
Pets                        100
Religion & Spirituality     100
Careers                     100
Shopping                     92
Name: tier1, dtype: int64

In [5]:
train_data['tier2'].value_counts()

Computing                   700
Travel Type                 478
Womens Fashion              444
Industries                  400
Business                    300
                           ... 
Romance Movies               61
Crime and Mystery Movies     59
Model Toys                   59
Extreme Sports               20
Golf                         17
Name: tier2, Length: 63, dtype: int64

In [6]:
train_data['tier3'].value_counts()

Internet                              400
Womens Clothing                       244
Endocrine and Metabolic Diseases      200
Motor Insurance                       200
Rail Travel                           150
Pharmaceutical Industry               100
Mens Clothing                         100
Adoption and Fostering                100
Business Travel                       100
Apparel Industry                      100
Business Accounting & Finance         100
Computer Peripherals                  100
Consumer Issues                       100
Credit Cards                          100
Van                                   100
Computer Software and Applications    100
Reproductive Health                   100
Housing Market                        100
Womens Shoes and Footwear             100
Womens Accessories                    100
Family Travel                         100
Business Banking & Finance            100
Defense Industry                      100
Classic Cars                      

In [7]:
train_data['tier4'].value_counts()

Womens Casual Wear            200
Web Hosting                   100
Internet of Things            100
IT and Internet Support       100
Recalls                       100
Shareware and Freeware        100
Hormonal Disorders            100
Mens Business Wear            100
Menopause                     100
Womens Jewelry and Watches    100
Web Conferencing              100
Infertility                   100
Venture Capital               100
Womens Sportswear              44
Name: tier4, dtype: int64

In [86]:
# put a limit on maximal number of features and minimal word frequency
tf_idf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', ngram_range=(1, 2))#TfidfVectorizer(max_features=50000,min_df=1)

# multinomial logistic regression a.k.a softmax classifier
logit = LogisticRegression(C=1e2, n_jobs=4, solver='lbfgs', 
                           random_state=42, verbose=0, 
                           multi_class='multinomial')
svm=OneClassSVM(gamma='scale')


In [129]:
from sklearn.svm import OneClassSVM
class Hierarchical(BaseEstimator):
    
    def __init__(self, 
                 base_model, 
                 base_model1,
                 model_store_path,
                 class_separator = '/',
                 min_size_to_train=50#tried 20 too
                ):

        self.base_model = base_model
        self.base_model1=base_model1
        self.model_store_path = Path(model_store_path)
        self.class_separator = class_separator
        self.min_size_to_train = min_size_to_train
        
        self.model_store_path.mkdir(exist_ok=True)
        
    def fit(self, X, y):
        
        lev1_classes = [label.split(self.class_separator)[0]
                        for label in y]
        
        with timer('Training level 1 model'):
            self.base_model.fit(X, lev1_classes)
            
            
            with open(self.model_store_path / 'level1_model.pkl', 'wb') as f:
                pickle.dump(self.base_model, f)
        
        
        for lev1_class in np.unique(lev1_classes):
            with timer(f'Training level 2 model for parent: {lev1_class}'):
                curr_X = X.loc[y.str.startswith(lev1_class)]
                curr_y = y.loc[y.str.startswith(lev1_class)].apply(lambda s: s.split(self.class_separator)[1]).str.strip()
                curr_y=curr_y[curr_y!=""]
                curr_X=X.loc[curr_y.index]
                if len(curr_X) < self.min_size_to_train:
                    print(f"Skipped class {lev1_class.replace(' ', '_')}as less data for further classification: {len(curr_X)}")
                    continue
                model_name = f"level2_model_{lev1_class.replace(' ', '_')}.pkl"
                try:
                    self.base_model.fit(curr_X, curr_y)                                  
                    with open(self.model_store_path / model_name, 'wb') as f:
                        pickle.dump(self.base_model, f)
                except ValueError:
                    print(np.unique(curr_y),"in", lev1_class)
                    #continue
                    self.base_model1.fit(curr_X, curr_y)                                    
                    with open(self.model_store_path / model_name, 'wb') as f:
                        pickle.dump(self.base_model1, f)
                

                    
        lev2_classes = [label.split(self.class_separator)[1]
                        for label in y]
        
        for lev2_class in np.unique(lev2_classes):
            if lev2_class=="":
                continue
            with timer(f'Training level 3 model for parent: {lev2_class}'):
                curr_X = X.loc[y.str.contains(lev2_class)]
                curr_y = y.loc[y.str.contains(lev2_class)].apply(lambda s: s.split(self.class_separator)[2]).str.strip()
                curr_y=curr_y[curr_y!=""]
                curr_X=X.loc[curr_y.index]
                if len(curr_X) < self.min_size_to_train:
                    print(f"Skipped class {lev2_class.replace(' ', '_')} as less data for further classification: {len(curr_X)}")
                    continue
                model_name = f"level3_model_{lev2_class.replace(' ', '_')}.pkl"
                try:
                    self.base_model.fit(curr_X, curr_y)                                  
                    with open(self.model_store_path / model_name, 'wb') as f:
                        pickle.dump(self.base_model, f)
                except ValueError:
                    print(np.unique(curr_y),"in", lev2_class)
                    self.base_model1.fit(curr_X, curr_y)                                    
                    with open(self.model_store_path / model_name, 'wb') as f:
                        pickle.dump(self.base_model1, f)

        lev3_classes = [label.split(self.class_separator)[2]
                        for label in y]
        
        for lev3_class in np.unique(lev3_classes):
            if lev3_class=="":
                continue
            with timer(f'Training level 4 model for parent: {lev3_class}'):
                curr_X = X.loc[y.str.contains(lev3_class)]
                curr_y = y.loc[y.str.contains(lev3_class)].apply(lambda s: s.split(self.class_separator)[3]).str.strip()
                curr_y=curr_y[curr_y!=""]
                curr_X=X.loc[curr_y.index]
                if len(curr_X) < self.min_size_to_train:
                    print(f"Skipped class {lev2_class.replace(' ', '_')} as less data for further classification: {len(curr_X)}")
                    continue
                model_name = f"level4_model_{lev3_class.replace(' ', '_')}.pkl"
                try:
                    self.base_model.fit(curr_X, curr_y)                                  
                    with open(self.model_store_path / model_name, 'wb') as f:
                        pickle.dump(self.base_model, f)
                except ValueError:
                    print(np.unique(curr_y),"in", lev3_class)
                    self.base_model1.fit(curr_X, curr_y)                                    
                    with open(self.model_store_path / model_name, 'wb') as f:
                        pickle.dump(self.base_model1, f)
    def predict(self, X):
        
        model_name =  'level1_model.pkl'
        with open(self.model_store_path / model_name, 'rb') as f:
            level1_model = pickle.load(f)
        
        level1_preds = level1_model.predict(X)
            
        level2_preds = np.zeros_like(level1_preds)
        level2_preds = np.array(level2_preds, dtype='object') 
            
        for lev1_class in np.unique(level1_preds):
            #print(lev1_class)
            idx = level1_preds == lev1_class
            curr_X = X.iloc[idx]
            
            model_name = f"level2_model_{lev1_class.replace(' ', '_')}.pkl"
            
            if Path(self.model_store_path / model_name).exists():
            
                with open(self.model_store_path / model_name, 'rb') as f:
                    level2_model = pickle.load(f)

                curr_level2_preds = level2_model.predict(curr_X)
                level2_preds[idx] = curr_level2_preds
            
            else:
                level2_preds[idx] = lev1_class
        
        level3_preds = np.zeros_like(level2_preds)
        level3_preds = np.array(level3_preds, dtype='object') 
        level2_preds=level2_preds.astype(str)
        for lev2_class in np.unique(level2_preds):
            idx = level2_preds == lev2_class
            curr_X = X.iloc[idx]       
            if lev2_class=="":
                level3_preds[idx]=""
                continue
            model_name = f"level3_model_{lev2_class.replace(' ', '_')}.pkl"
            #print("tier3",model_name)
            if Path(self.model_store_path / model_name).exists():
                #print("tier3",model_name)
                with open(self.model_store_path / model_name, 'rb') as f:
                    level3_model = pickle.load(f)

                curr_level3_preds = level3_model.predict(curr_X)
                level3_preds[idx] = curr_level3_preds
            
            else:
                level3_preds[idx] = lev2_class
            
        level4_preds = np.zeros_like(level3_preds)
        level4_preds = np.array(level4_preds, dtype='object') 
        level3_preds=level3_preds.astype(str)   
        for lev3_class in np.unique(level3_preds):
            idx = level3_preds == lev3_class
            curr_X = X.iloc[idx]       
            if lev3_class=="":
                level4_preds[idx]=""
                continue
            model_name = f"level4_model_{lev3_class.replace(' ', '_')}.pkl"
            #print("tier3",model_name)
            if Path(self.model_store_path / model_name).exists():
                #print("tier4",model_name)
                with open(self.model_store_path / model_name, 'rb') as f:
                    level4_model = pickle.load(f)

                curr_level4_preds = level4_model.predict(curr_X)
                level4_preds[idx] = curr_level4_preds
            
            else:
                level4_preds[idx] = lev3_class
        
                
        return level1_preds, level2_preds,level3_preds,level4_preds

In [11]:
from sklearn.compose import make_column_transformer

feature exploration

In [12]:
train_data.columns

Index(['index', 'id', 'content', 'title', 'preprocessed_content', 'label',
       'hierarchy', 'hierarchy_#tiers', 'tier1', 'tier2', 'tier3', 'tier4'],
      dtype='object')

In [14]:
train_data=train_data.drop(['index'],axis=1)

In [13]:
import numpy as np
train_data['check_publisher_id']=np.NaN
train_data['publisher_id']=np.NaN
def publisher_id(x):  
    pub=x['id'].split('/')
    #print(pub)
    if len(pub)>1:
#         x['check_publisher_id']=0
#         x['publisher_id']=pub[2]
        try:
            return pd.Series((0,pub[2]))
        except IndexError:
            return pd.Series((0,pub[1]))
    else:
#         x['check_publisher_id']=1
#         x['publisher_id']=pub
        return pd.Series((1,pub[0]))
train_data[['check_publisher_id','publisher_id']]=train_data.apply(publisher_id,axis=1)


In [14]:
train_data['publisher_id'].nunique()

457

In [16]:
def chk_pub_id(x):
    if x['check_publisher_id']==1:
        l=x['publisher_id'].split('_')
        return pd.Series(l[0])
    else:
        return pd.Series(x['publisher_id'])
train_data['publisher_id']=train_data.apply(chk_pub_id,axis=1)    

In [17]:
train_data['publisher_id'].nunique()

363

In [18]:
pd.isna(train_data).sum()

index                      0
id                         0
content                  158
title                    222
preprocessed_content     175
label                      0
hierarchy                  0
hierarchy_#tiers           0
tier1                      0
tier2                    492
tier3                   5091
tier4                   7723
check_publisher_id         0
publisher_id               0
dtype: int64

In [20]:
def length(x):
    if not pd.isna(x):
        return (len(x.split(" ")))
train_data['length_preprocessed_content']=train_data['preprocessed_content'].apply(length)

In [21]:
train_data['length_preprocessed_content'].describe()

count    8992.000000
mean      433.420262
std       400.555787
min         1.000000
25%       177.000000
50%       334.000000
75%       568.000000
max      6528.000000
Name: length_preprocessed_content, dtype: float64

new train and test data

In [22]:
len(train_data)

9167

In [23]:
train_data=train_data.fillna('')

In [24]:
train_data['tier1']=train_data['tier1'].str.strip()
train_data['tier2']=train_data['tier2'].str.strip()
train_data['tier3']=train_data['tier3'].str.strip()
train_data['tier4']=train_data['tier4'].str.strip()

In [38]:
train_data['tier1_tier2_tier3_tier4'] = train_data['tier1']+ '/' + train_data['tier2']+ '/' + train_data['tier3']+'/' + train_data['tier4']
train_data.head()

,index,id,content,title,preprocessed_content,label,hierarchy,hierarchy_#tiers,tier1,tier2,tier3,tier4,check_publisher_id,publisher_id,length_preprocessed_content,tier1_tier2_tier3_tier4
0,133,https://www.whowhatwear.com/5-looks-that-put-b...,After seeing burgundy boots on some of our fav...,5 Looks That Put Burgundy Boots On Our Must-Ha...,seeing burgundy boot favorite street style blo...,Womens\ Casual\ Wear,"[{'Style & Fashion': {""Women's Fashion"": {""Wom...",4,Style & Fashion,Womens Fashion,Womens Clothing,Womens Casual Wear,0,www.whowhatwear.com,31,Style & Fashion/Womens Fashion/Womens Clothing...
1,189,https://www.wisebread.com/what-you-need-to-kno...,"According to a September 29, 2009 Fidelity Inv...",How to Make the Most of Your 401K,according september fidelity investments study...,Personal\ Investing,[{'Personal Finance': 'Personal Investing'}],2,Personal Finance,Personal Investing,,,0,www.wisebread.com,778,Personal Finance/Personal Investing//
2,401,https://www.theturekclinic.com/blog/mindful-ea...,"“Thou shouldst eat to live; not live to eat,” ...",Mindful Eating,thou shouldst eat live live eat said socrates ...,Mens\ Health,"[{'Healthy Living': ""Men's Health""}]",2,Healthy Living,Mens Health,,,0,www.theturekclinic.com,363,Healthy Living/Mens Health//
3,198,https://www.news-medical.net/news/20190611/Ear...,Researchers continue to dig for molecular clue...,Early life adversity and high levels of FKBP5 ...,researchers continue dig molecular clue better...,Hormonal\ Disorders,[{'Medical Health': {'Diseases and Conditions'...,4,Medical Health,Diseases and Conditions,Endocrine and Metabolic Diseases,Hormonal Disorders,0,www.news-medical.net,356,Medical Health/Diseases and Conditions/Endocri...
4,24,https://hobbylark.com/card-games/Top-Strongest...,"As any duelist knows, the vast majority of Yu-...",Top 10 Strongest (Highest ATK) Monsters in Yu-...,duelist know vast majority yu gi oh match end ...,Card\ Games,[{'Hobbies & Interests': {'Games and Puzzles':...,3,Hobbies & Interests,Games and Puzzles,Card Games,,0,hobbylark.com,63,Hobbies & Interests/Games and Puzzles/Card Games/


In [39]:
train_data['tier1_tier2_tier3_tier4'].value_counts().head()

Sports/Cricket//                                                     200
Education/Homework and Study//                                       200
Personal Finance/Insurance/Motor Insurance/                          199
Style & Fashion/Womens Fashion/Womens Clothing/Womens Casual Wear    195
Travel/Travel Type/Rail Travel/                                      142
Name: tier1_tier2_tier3_tier4, dtype: int64

In [198]:
c=train_data[train_data['tier1_tier2_tier3_tier4'].str.contains('Career')]
c.head()

,index,id,content,title,preprocessed_content,label,hierarchy,hierarchy_#tiers,tier1,tier2,tier3,tier4,check_publisher_id,publisher_id,length_preprocessed_content,tier1_tier2_tier3_tier4
247,124,https://www.reed.co.uk/career-advice/how-to-fi...,"by Amber RolfeFor many recent graduates, findi...",How to find: international graduate jobs,amber rolfefor many recent graduate finding jo...,Career\ Advice,[{'Careers': 'Career Advice'}],2,Careers,Career Advice,,,0,www.reed.co.uk,531,Careers/Career Advice//
301,309,https://www.reed.co.uk/career-advice/what-can-...,by Amber RolfeReady to put your career in the ...,What can I do with my law degree?,amber rolfeready put career hand law got cover...,Career\ Advice,[{'Careers': 'Career Advice'}],2,Careers,Career Advice,,,0,www.reed.co.uk,608,Careers/Career Advice//
330,704,https://www.reed.co.uk/career-advice/mature-st...,by Amber RolfeThe prospect of entering univers...,Mature students: What you need to know,amber rolfethe prospect entering university ma...,Career\ Advice,[{'Careers': 'Career Advice'}],2,Careers,Career Advice,,,0,www.reed.co.uk,468,Careers/Career Advice//
337,882,https://www.reed.co.uk/career-advice/the-top-c...,"by Michael ChearyOk, so aside from terrible St...",The top courses to help get you hired May 2015,michael chearyok aside terrible star wars rela...,Career\ Advice,[{'Careers': 'Career Advice'}],2,Careers,Career Advice,,,0,www.reed.co.uk,381,Careers/Career Advice//
477,796,https://www.reed.co.uk/career-advice/what-its-...,by Michael ChearyConsidering working in the ch...,What it’s really like to work in the charity s...,michael chearyconsidering working charity sect...,Career\ Advice,[{'Careers': 'Career Advice'}],2,Careers,Career Advice,,,0,www.reed.co.uk,524,Careers/Career Advice//


In [199]:
c['tier2'].unique()

array(['Career Advice'], dtype=object)

In [40]:
r1=train_data[train_data['tier3'].str.contains("'s")]['tier3'].index
train_data.loc[r1,'tier3']=train_data.loc[r1,'tier3'].str.replace("'s","s")

r2=train_data[train_data['tier2'].str.contains("'s")]['tier2'].index
train_data.loc[r2,'tier2']=train_data.loc[r2,'tier2'].str.replace("'s","s")

train_data['tier1']=train_data['tier1'].str.strip()
train_data['tier2']=train_data['tier2'].str.strip()
train_data['tier3']=train_data['tier3'].str.strip()
train_data['tier4']=train_data['tier4'].str.strip()

train_data['tier2']=train_data['tier2'].str.strip("\"")
train_data['tier3']=train_data['tier3'].str.strip("\"")
train_data['tier4']=train_data['tier4'].str.strip("\"")

In [41]:
train_data.columns

Index(['index', 'id', 'content', 'title', 'preprocessed_content', 'label',
       'hierarchy', 'hierarchy_#tiers', 'tier1', 'tier2', 'tier3', 'tier4',
       'check_publisher_id', 'publisher_id', 'length_preprocessed_content',
       'tier1_tier2_tier3_tier4'],
      dtype='object')

In [42]:
ind=train_data[train_data['title']==""].index
ind1=train_data[train_data['preprocessed_content']==""].index

In [43]:
train_data=train_data[~(train_data.index.isin(ind))]
train_data=train_data[~(train_data.index.isin(ind1))]
len(train_data)

8774

In [44]:
features1=train_data[['title','preprocessed_content','publisher_id','length_preprocessed_content']]#,'tier1','tier2','tier3','tier4']]
labels1=train_data[['tier1', 'tier2', 'tier3', 'tier4', 'tier1_tier2_tier3_tier4']]
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(features1, labels1,test_size=0.2,random_state=42,stratify=labels1['tier4'])

In [45]:
X_train_new = X_train_new.reset_index(drop=True)
y_train_new = y_train_new.reset_index(drop=True)
X_test_new = X_test_new.reset_index(drop=True)
y_test_new = y_test_new.reset_index(drop=True)

without embedding and other features(length of description and publisher_id)

In [46]:
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
combine = make_column_transformer((tf_idf,'title'),(tf_idf,'preprocessed_content'),(StandardScaler(),['length_preprocessed_content']),(OneHotEncoder(handle_unknown='ignore',sparse=False),['publisher_id']),remainder='passthrough')

In [130]:
base_model = Pipeline([('combine', combine), 
                       ('logit', logit)])
base_model1 = Pipeline([('combine', combine), 
                       ('svm', svm)])
model = Hierarchical(
    base_model=base_model,
    base_model1=base_model1,
    model_store_path='modelsnew'
)

model.fit(X_train_new,y_train_new['tier1_tier2_tier3_tier4'])

[Training level 1 model] done in 66 s
[Training level 2 model for parent: Automotive] done in 1 s
[Training level 2 model for parent: Books and Literature] done in 1 s
[Training level 2 model for parent: Business and Finance] done in 2 s
['Career Advice'] in Careers
[Training level 2 model for parent: Careers] done in 0 s
[Training level 2 model for parent: Education] done in 1 s
[Training level 2 model for parent: Events and Attractions] done in 0 s
['Parenting'] in Family and Relationships
[Training level 2 model for parent: Family and Relationships] done in 0 s
['Desserts and Baking'] in Food & Drink
[Training level 2 model for parent: Food & Drink] done in 0 s
[Training level 2 model for parent: Healthy Living] done in 0 s
[Training level 2 model for parent: Hobbies & Interests] done in 1 s
[Training level 2 model for parent: Medical Health] done in 1 s
[Training level 2 model for parent: Movies] done in 2 s
[Training level 2 model for parent: Music and Audio] done in 1 s
['Disaste

['Africa Travel'] in Travel Locations
[Training level 3 model for parent: Travel Locations] done in 0 s
[Training level 3 model for parent: Travel Type] done in 1 s
Skipped class Volleyball as less data for further classification: 0
[Training level 3 model for parent: Volleyball] done in 0 s
Skipped class Water_Polo as less data for further classification: 0
[Training level 3 model for parent: Water Polo] done in 0 s
[Training level 3 model for parent: Womens Fashion] done in 1 s
Skipped class Womens_Health as less data for further classification: 0
[Training level 3 model for parent: Womens Health] done in 0 s
Skipped class Womens_Health as less data for further classification: 0
[Training level 4 model for parent: Adoption and Fostering] done in 0 s
Skipped class Womens_Health as less data for further classification: 0
[Training level 4 model for parent: Africa Travel] done in 0 s
Skipped class Womens_Health as less data for further classification: 0
[Training level 4 model for paren

In [131]:
level1_pred, level2_pred, level3_pred, level4_pred= model.predict(X_test_new)

In [143]:
finalresult=pd.concat([X_test_new,y_test_new['tier1_tier2_tier3_tier4'],y_test_new['tier1'],y_test_new['tier2'],y_test_new['tier3'],y_test_new['tier4'],pd.DataFrame(level1_pred,columns=['pred1']),pd.DataFrame(level2_pred,columns=['pred2']),pd.DataFrame(level3_pred,columns=['pred3']),pd.DataFrame(level4_pred,columns=['pred4'])],axis=1)

In [145]:
oneclass=['Career Advice','Parenting','Desserts and Baking','Disasters','Sikhism','Womens Fashion','Computing','Van','Classic Cars','Audio Production','Standardized Testing','Motor Insurance','Adoption and Fostering','Credit Cards','Mutual Funds','Africa Travel','Venture Capital','Shareware and Freeware','Recalls','Infertility','Womens Jewelry and Watches']

In [144]:
finalresult

,title,preprocessed_content,publisher_id,length_preprocessed_content,tier1_tier2_tier3_tier4,tier1,tier2,tier3,tier4,pred1,pred2,pred3,pred4
0,Menopause is not the end of women&#39;s love life,menopause signal end woman love life fact woma...,www.news-medical.net,225,Medical Health/Diseases and Conditions/Endocri...,Medical Health,Diseases and Conditions,Endocrine and Metabolic Diseases,Menopause,Medical Health,Diseases and Conditions,Endocrine and Metabolic Diseases,Menopause
1,BREAKING: Tomorrowland Winter 2020 Is Official...,french government shutting tomorrowland winter...,www.youredm.com,125,Music and Audio/Dance and Electronic Music//,Music and Audio,Dance and Electronic Music,,,Music and Audio,Dance and Electronic Music,Dance and Electronic Music,Dance and Electronic Music
2,Credit Cards Stoke JPMorgan Comeback,credit card well jpm jpmorgan chase one leadin...,www.forbes.com,490,Personal Finance/Personal Debt/Credit Cards/,Personal Finance,Personal Debt,Credit Cards,,Personal Finance,Personal Debt,-1,-1
3,1993 Nissan President Is The RWD Flagship You ...,japanese domestic market jdm car bee knee dist...,www.motor1.com,156,Automotive/Auto Type/Classic Cars/,Automotive,Auto Type,Classic Cars,,Automotive,Auto Type,-1,-1
4,Immaculate Car Collection In Mexico Is Eclecti...,cool car universal language passion collecting...,www.motor1.com,204,Automotive/Auto Type/Classic Cars/,Automotive,Auto Type,Classic Cars,,Automotive,Auto Type,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1750,How Georgia Satellites' 'Keep Your Hands to Yo...,thirty year ago georgia satellites released on...,api.swiftype.com,884,Music and Audio/Country Music//,Music and Audio,Country Music,,,Music and Audio,Country Music,Country Music,Country Music
1751,The Farm Bill works for birds — and needs your...,farmland devoid bird certain part farmer land ...,www.birdwatchingdaily.com,420,Hobbies & Interests/Birdwatching//,Hobbies & Interests,Birdwatching,,,Hobbies & Interests,Birdwatching,Birdwatching,Birdwatching
1752,Adoption Champions,adoption become widely discussed supported eve...,www.angeladoptioninc.com,307,Family and Relationships/Parenting/Adoption an...,Family and Relationships,Parenting,Adoption and Fostering,,Family and Relationships,-1,-1,-1
1753,Poster poems: seeking refuge and asylum,channel tunnel besieged dispossessed mediterra...,www.theguardian.com,378,Books and Literature/Poetry//,Books and Literature,Poetry,,,Books and Literature,Poetry,Poetry,Poetry


In [137]:
finalresult.loc[0,'tier1_tier2_tier3_tier4']

'Medical Health/Diseases and Conditions/Endocrine and Metabolic Diseases/Menopause'

In [147]:
finalresult[(finalresult['pred2'].str.contains('1'))|(finalresult['pred3'].str.contains('1'))]

,title,preprocessed_content,publisher_id,length_preprocessed_content,tier1_tier2_tier3_tier4,tier1,tier2,tier3,tier4,pred1,pred2,pred3,pred4
2,Credit Cards Stoke JPMorgan Comeback,credit card well jpm jpmorgan chase one leadin...,www.forbes.com,490,Personal Finance/Personal Debt/Credit Cards/,Personal Finance,Personal Debt,Credit Cards,,Personal Finance,Personal Debt,-1,-1
3,1993 Nissan President Is The RWD Flagship You ...,japanese domestic market jdm car bee knee dist...,www.motor1.com,156,Automotive/Auto Type/Classic Cars/,Automotive,Auto Type,Classic Cars,,Automotive,Auto Type,-1,-1
4,Immaculate Car Collection In Mexico Is Eclecti...,cool car universal language passion collecting...,www.motor1.com,204,Automotive/Auto Type/Classic Cars/,Automotive,Auto Type,Classic Cars,,Automotive,Auto Type,1,1
6,New ASUS ExpertBook A Solid Work Laptop,asus expertbook b9450 first thing notice new a...,www.forbes.com,608,Technology & Computing/Computing/Laptops/,Technology & Computing,Computing,Laptops,,Technology & Computing,1,1,1
8,Are Desktop PCs And Laptops Becoming Obsolete?,desktop pcs laptop rendered obsolete originall...,www.forbes.com,524,Technology & Computing/Computing/Laptops/,Technology & Computing,Computing,Laptops,,Technology & Computing,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1743,Recipe Of The Day: Corn Soup,summer corn peak hard want eat way right cob c...,www.huffingtonpost.com,69,Food & Drink///,Food & Drink,,,,Food & Drink,-1,-1,-1
1744,"Cloud Flash, Standard Tests And Storage Versus...",enterprise data volume doubling every two year...,www.forbes.com,962,Education/Educational Assessment/Standardized ...,Education,Educational Assessment,Standardized Testing,,Technology & Computing,1,1,1
1748,The top companies hiring in June 2016,michael chearyso british summer time finally c...,www.reed.co.uk,540,Careers/Career Advice//,Careers,Career Advice,,,Careers,-1,-1,-1
1749,XAG Robot Joins Drone Fleet to Initiate Ground...,guangzhou china feb prnewswire since emergence...,www.prnewswire.com,686,Business and Finance/Industries/Defense Industry/,Business and Finance,Industries,Defense Industry,,Technology & Computing,-1,-1,-1


LEVEL 1 PREDICTIONS

In [148]:
#misclassifications
finalresult[finalresult['tier1']!=finalresult['pred1']]

,title,preprocessed_content,publisher_id,length_preprocessed_content,tier1_tier2_tier3_tier4,tier1,tier2,tier3,tier4,pred1,pred2,pred3,pred4
148,Defense™ by X-Doria Launches Fast Charging Dua...,santa monica calif oct prnewswire defense x do...,www.prnewswire.com,245,Technology & Computing/Computing/Computer Peri...,Technology & Computing,Computing,Computer Peripherals,,Business and Finance,Industries,Defense Industry,Defense Industry
170,Trendlines Agrifood Fund Makes First Investment,misgav israel singapore feb prnewswire trendli...,www.prnewswire.com,439,Business and Finance/Business/Business Banking...,Business and Finance,Business,Business Banking & Finance,Venture Capital,Food & Drink,-1,-1,-1
183,Standard Chartered Drops On Cost Concerns,standard chartered may beat analyst expectatio...,www.forbes.com,374,Education/Educational Assessment/Standardized ...,Education,Educational Assessment,Standardized Testing,,Personal Finance,Insurance,-1,-1
200,Moldavská národní asociace cestovního ruchu: 3...,ki v moldavsko b ezna prnewswire moldavsk repu...,www.prnewswire.com,434,Events and Attractions/Amusement and Theme Par...,Events and Attractions,Amusement and Theme Parks,,,Business and Finance,Industries,Pharmaceutical Industry,Pharmaceutical Industry
283,Burn Survivor Celebrates Second Pregnancy with...,one burn survivor defying odds one burn surviv...,www.people.com,354,Pop Culture/Celebrity Pregnancy//,Pop Culture,Celebrity Pregnancy,,,Education,Homework and Study,Homework and Study,Homework and Study
307,Testing Corporate 'Greenness',recent report issued new scientist shown fasci...,www.forbes.com,450,Education/Educational Assessment/Standardized ...,Education,Educational Assessment,Standardized Testing,,Technology & Computing,1,1,1
381,Miss North Dakota Cara Mund Is Crowned State's...,first time competition history miss north dako...,www.huffingtonpost.com,145,Style & Fashion///,Style & Fashion,,,,Food & Drink,-1,-1,-1
408,PressReader ha firmado un acuerdo de licencia ...,la nueva iniciativa har que la r plica digital...,www.prnewswire.com,850,Technology & Computing/Computing/Internet/IT a...,Technology & Computing,Computing,Internet,IT and Internet Support,Business and Finance,Business,Business Banking & Finance,-1
431,"Go, Frankenfish! Why We Need GM Salmon.",trust heard genetically modified salmon curren...,www.huffingtonpost.com,296,Food & Drink///,Food & Drink,,,,Family and Relationships,1,1,1
474,Disney Just Revealed New Images of the Star Wa...,new detail world time christmas creatives walt...,www.travelandleisure.com,241,Events and Attractions/Amusement and Theme Par...,Events and Attractions,Amusement and Theme Parks,,,Travel,Travel Type,Family Travel,Family Travel


In [150]:
print("Accuracy of level 1",1-len(finalresult[finalresult['tier1']!=finalresult['pred1']])/len(finalresult))

Accuracy of level 1 0.9766381766381766


LEVEL 2 PREDICTIONS

In [160]:
lev2=finalresult[(finalresult['tier2']!=finalresult['pred2'])]
a=lev2[(lev2['tier2'].isin(oneclass))&(lev2['pred2']=='-1')]

In [168]:
b=lev2[~(lev2.index.isin(a.index))&(lev2['pred2']!='1')]

In [169]:
print("Accuracy of level 2",1-(len(a)+len(b))/len(finalresult))

Accuracy of level 2 0.8227920227920228


LEVEL 3 PREDCITIONS

In [178]:
lev3=finalresult[(finalresult['tier3']!=finalresult['pred3'])&(finalresult['tier3']!="")]

In [179]:
a1=lev3[(lev3['tier3'].isin(oneclass))&(lev3['pred3']=='-1')]

In [180]:
b1=lev3[~(lev3.index.isin(a1.index))&(lev3['pred3']!='1')]

In [181]:
print("Accuracy of level 3",1-(len(a1)+len(b1))/len(finalresult))

Accuracy of level 3 0.8478632478632478


LEVEL 4 PREDICTIONS

In [182]:
lev4=finalresult[(finalresult['tier4']!=finalresult['pred4'])&(finalresult['tier4']!="")]
lev4

,title,preprocessed_content,publisher_id,length_preprocessed_content,tier1_tier2_tier3_tier4,tier1,tier2,tier3,tier4,pred1,pred2,pred3,pred4
7,The Normalization of Mean Leaders: A Recipe fo...,many u wondering glory day kalanicks shkrelis ...,www.entrepreneur.com,824,Business and Finance/Business/Business Banking...,Business and Finance,Business,Business Banking & Finance,Venture Capital,Business and Finance,Business,Business Banking & Finance,-1
32,Mindtree von ISG zu einem Marktführer für Digi...,unternehmen f r einschneidendes design thinkin...,www.prnewswire.com,701,Technology & Computing/Computing/Internet/IT a...,Technology & Computing,Computing,Internet,IT and Internet Support,Technology & Computing,-1,-1,-1
34,Breast cancer medication letrozole could incre...,medicine used breast cancer treatment consider...,www.news-medical.net,344,Medical Health/Diseases and Conditions/Reprodu...,Medical Health,Diseases and Conditions,Reproductive Health,Infertility,Medical Health,Diseases and Conditions,Reproductive Health,1
41,7 Comfy Vacation Outfit Ideas That Won't Make ...,packing vacation sometimes feel like pick dres...,www.whowhatwear.com,139,Style & Fashion/Womens Fashion/Womens Clothing...,Style & Fashion,Womens Fashion,Womens Clothing,Womens Casual Wear,Style & Fashion,1,1,1
55,17 Reasons to Finally Invest in a Leather Jack...,every season try beautiful leather jacket look...,www.whowhatwear.com,80,Style & Fashion/Womens Fashion/Womens Clothing...,Style & Fashion,Womens Fashion,Womens Clothing,Womens Casual Wear,Style & Fashion,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,The J. M. Smucker Company Issues Voluntary Rec...,orrville ohio dec prnewswire j smucker company...,www.prnewswire.com,162,Business and Finance/Business/Consumer Issues/...,Business and Finance,Business,Consumer Issues,Recalls,Business and Finance,Business,Consumer Issues,-1
1718,Thrillist's Ben Lerer on Success as a Young Trep,year old ben lerer proven sweet spot medium co...,www.entrepreneur.com,381,Business and Finance/Business/Business Banking...,Business and Finance,Business,Business Banking & Finance,Venture Capital,Business and Finance,Business,Business Banking & Finance,1
1737,Why CIOs Need To Think About The Internet Of T...,stunning example provincialism washington post...,www.forbes.com,808,Technology & Computing/Computing/Internet/Inte...,Technology & Computing,Computing,Internet,Internet of Things,Technology & Computing,1,1,1
1740,Why Venture Capitalists Heart NY,big apple recently outpaced massachusetts fame...,www.entrepreneur.com,208,Business and Finance/Business/Business Banking...,Business and Finance,Business,Business Banking & Finance,Venture Capital,Business and Finance,Business,Business Banking & Finance,-1


In [184]:
a2=lev4[(lev4['tier4'].isin(oneclass))&(lev4['pred4']=='-1')]

In [186]:
b2=lev4[~(lev4.index.isin(a2.index))&(lev4['pred4']!='1')]

In [187]:
print("Accuracy of level 4",1-(len(a2)+len(b2))/len(finalresult))

Accuracy of level 4 0.9156695156695157


Total accuracy 0.8855